# Udemy Data Cleaning
Cleaning of the raw data pulled from Udemy is required

In [1]:
# Library 
import pandas as pd 
import numpy as np 
from ast import literal_eval
import re
import matplotlib.pyplot as plt

## Import Data

In [2]:
data = pd.read_csv("./Data/Udemy_Raw.csv")
# Inspect dataframe
data.head()

,Unnamed: 0,ID,Title,Overall_Rating,Best_Rating,Worst_Rating,No_of_Ratings,Category,Subcategory,Topic,...,Language,SkillsFuture,No_of_Practice_Test,No_of_Articles,No_of_Coding_Exercises,Video_Duration,No_of_Additional_Resources,Bestseller,Price,Discounted_Price
0,0,1236468,Complete Hypnotherapy & Hypnosis Certification...,4.7,5,0.5,3524,Lifestyle,Esoteric Practices,Hypnotherapy,...,English,False,0,4,0,3 hours,0,Yes,104.98,"{'amount': 26.0, 'currency': 'SGD', 'price_str..."
1,0,3544722,Pinterest Marketing for Wedding Professionals ...,5.0,5,0.5,1,Marketing,Social Media Marketing,Pinterest Marketing,...,English,False,0,0,0,36 mins,2,No,29.98,"{'amount': 22.0, 'currency': 'SGD', 'price_str..."
2,0,3075744,Master the Telephone Sales- Cold calling Secrets,4.5,5,0.5,3,Marketing,Product Marketing,Marketing Strategy,...,English,False,0,0,0,44 mins,0,No,29.98,"{'amount': 22.0, 'currency': 'SGD', 'price_str..."
3,0,3732596,5 Practical Management concepts you MUST know,5.0,5,0.5,2,Personal Development,Leadership,Management Skills,...,English,False,0,0,0,2 hours,0,No,49.98,"{'amount': 22.0, 'currency': 'SGD', 'price_str..."
4,0,745438,Fermented Foods Mastery,4.5,5,0.5,187,Health & Fitness,Nutrition,Fermented Foods,...,English,False,0,3,0,1.5 hours,12,No,68.98,"{'amount': 24.0, 'currency': 'SGD', 'price_str..."


In [3]:
# Summary
data.describe()

,Unnamed: 0,ID,Overall_Rating,Best_Rating,Worst_Rating,No_of_Ratings,No_of_Practice_Test,No_of_Articles,No_of_Coding_Exercises,No_of_Additional_Resources,Price
count,16429.0,1.642900e+04,16429.000000,16429.000000,16429.000000,16429.000000,16429.000000,16429.000000,16429.000000,16429.000000,16429.00000
mean,0.0,2.598236e+06,3.850825,4.208412,0.420841,1045.104571,0.145718,2.628462,0.107432,9.195812,74.88489
std,0.0,1.361480e+06,1.720963,1.825248,0.182525,6405.752256,0.742840,9.123434,2.929279,29.134401,44.75246
min,0.0,7.723000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,0.0,1.333462e+06,4.100000,5.000000,0.500000,1.000000,0.000000,0.000000,0.000000,0.000000,29.98000
50%,0.0,2.944506e+06,4.500000,5.000000,0.500000,10.000000,0.000000,0.000000,0.000000,1.000000,59.98000
75%,0.0,3.925690e+06,4.900000,5.000000,0.500000,275.000000,0.000000,2.000000,0.000000,8.000000,128.98000
max,0.0,4.201364e+06,5.000000,5.000000,0.500000,374285.000000,6.000000,229.000000,201.000000,810.000000,218.98000


## Step 1: Removing irrelevant columns

In [4]:
# Dropping extra index and ID
data.drop(columns=["Unnamed: 0", "ID"], inplace = True)

## Step 2: Duplicates

In [5]:
# Checking for duplicates
data.duplicated().sum()

1

In [6]:
# Eyeballing duplicates to check for discrepancies
duplicated_title = data.loc[data.duplicated() == True, "Title"][7934]
data[data["Title"] == duplicated_title]

,Title,Overall_Rating,Best_Rating,Worst_Rating,No_of_Ratings,Category,Subcategory,Topic,Instructor,Language,SkillsFuture,No_of_Practice_Test,No_of_Articles,No_of_Coding_Exercises,Video_Duration,No_of_Additional_Resources,Bestseller,Price,Discounted_Price
4566,DIY Holiday Cards in Photoshop,5.0,5,0.5,1,Design,Design Tools,Photoshop,Khara Plicanic,English,False,0,0,0,1 hour,1,No,44.98,"{'amount': 22.0, 'currency': 'SGD', 'price_str..."
7888,DIY Holiday Cards in Photoshop,5.0,5,0.5,1,Design,Design Tools,Photoshop,Khara Plicanic,English,False,0,0,0,36 mins,1,No,44.98,"{'amount': 22.0, 'currency': 'SGD', 'price_str..."
7934,DIY Holiday Cards in Photoshop,5.0,5,0.5,1,Design,Design Tools,Photoshop,Khara Plicanic,English,False,0,0,0,1 hour,1,No,44.98,"{'amount': 22.0, 'currency': 'SGD', 'price_str..."


Upon checking, there are 3 different courses with the same title and same instructor. The 3 courses are for different seasons hence, we will rename them to ensure that they can be uniquely identified. 

In [7]:
# Manually changing titles for unique identification
data.loc[4566, "Title"] = "DIY Holiday Cards in Photoshop (Merry & Bright)"
data.loc[7888, "Title"] = "DIY Holiday Cards in Photoshop (Funky Design)"
data.loc[7934, "Title"] = "DIY Holiday Cards in Photoshop (Seasons of Love)"

# Checking if changes are effective
data.duplicated().sum()

0

## Step 3: Data Errors
During the web scrapping, there was some error in the extraction of the discounted price as the Udemy website is dynamic which resulted in a dictionary being stored for each row. Therefore now we have to extract the amount from each dictionary for each row.

In [8]:
# Function to extract discount amount from each row
def extract_amount(string): 
    # Convert string to dictionary
    dictionary = literal_eval(string)
    return dictionary["amount"]
    
# Apply function to every row
data["Discounted_Price"] = data.loc[data["Discounted_Price"].notna(), 
                                    "Discounted_Price"].apply(lambda x: extract_amount(x))
data.head()

,Title,Overall_Rating,Best_Rating,Worst_Rating,No_of_Ratings,Category,Subcategory,Topic,Instructor,Language,SkillsFuture,No_of_Practice_Test,No_of_Articles,No_of_Coding_Exercises,Video_Duration,No_of_Additional_Resources,Bestseller,Price,Discounted_Price
0,Complete Hypnotherapy & Hypnosis Certification...,4.7,5,0.5,3524,Lifestyle,Esoteric Practices,Hypnotherapy,Dr Karen E Wells,English,False,0,4,0,3 hours,0,Yes,104.98,26.0
1,Pinterest Marketing for Wedding Professionals ...,5.0,5,0.5,1,Marketing,Social Media Marketing,Pinterest Marketing,Staci Nichols,English,False,0,0,0,36 mins,2,No,29.98,22.0
2,Master the Telephone Sales- Cold calling Secrets,4.5,5,0.5,3,Marketing,Product Marketing,Marketing Strategy,Sanjay Bhasin,English,False,0,0,0,44 mins,0,No,29.98,22.0
3,5 Practical Management concepts you MUST know,5.0,5,0.5,2,Personal Development,Leadership,Management Skills,Vasudev Murthy,English,False,0,0,0,2 hours,0,No,49.98,22.0
4,Fermented Foods Mastery,4.5,5,0.5,187,Health & Fitness,Nutrition,Fermented Foods,Kale Brock,English,False,0,3,0,1.5 hours,12,No,68.98,24.0


## Step 4: Data Types

In [9]:
data.dtypes

Title                          object
Overall_Rating                float64
Best_Rating                     int64
Worst_Rating                  float64
No_of_Ratings                   int64
Category                       object
Subcategory                    object
Topic                          object
Instructor                     object
Language                       object
SkillsFuture                     bool
No_of_Practice_Test             int64
No_of_Articles                  int64
No_of_Coding_Exercises          int64
Video_Duration                 object
No_of_Additional_Resources      int64
Bestseller                     object
Price                         float64
Discounted_Price              float64
dtype: object

Overall the data types seem relatively suitable with the exception of "Video_Duration" which is the duration of course videos. Hence we will convert all duration to be in terms of hours. 

In [10]:
# Function to convert all duration to be in terms of hours
def hour_conversion(duration):
    
    # Copy straight if duration is in hours
    if duration.find("hour") != -1: 
        duration = re.sub(r"[a-z]", "", duration)
        duration = float(duration)
        
    # Convert to hours if duration is in mins
    elif duration.find("mins") != -1: 
        duration = re.sub(r"[a-z]", "", duration)
        duration = float(duration)/60
    
    return duration

# Apply conversion to dataframe
data["Video_Duration"] = data.loc[data["Video_Duration"].notna(), "Video_Duration"].apply(lambda x: hour_conversion(x))

# Rename column for clarity
data.rename(columns={"Video_Duration": "Video_Duration_Hr"}, inplace=True)

data.head()

,Title,Overall_Rating,Best_Rating,Worst_Rating,No_of_Ratings,Category,Subcategory,Topic,Instructor,Language,SkillsFuture,No_of_Practice_Test,No_of_Articles,No_of_Coding_Exercises,Video_Duration_Hr,No_of_Additional_Resources,Bestseller,Price,Discounted_Price
0,Complete Hypnotherapy & Hypnosis Certification...,4.7,5,0.5,3524,Lifestyle,Esoteric Practices,Hypnotherapy,Dr Karen E Wells,English,False,0,4,0,3.000000,0,Yes,104.98,26.0
1,Pinterest Marketing for Wedding Professionals ...,5.0,5,0.5,1,Marketing,Social Media Marketing,Pinterest Marketing,Staci Nichols,English,False,0,0,0,0.600000,2,No,29.98,22.0
2,Master the Telephone Sales- Cold calling Secrets,4.5,5,0.5,3,Marketing,Product Marketing,Marketing Strategy,Sanjay Bhasin,English,False,0,0,0,0.733333,0,No,29.98,22.0
3,5 Practical Management concepts you MUST know,5.0,5,0.5,2,Personal Development,Leadership,Management Skills,Vasudev Murthy,English,False,0,0,0,2.000000,0,No,49.98,22.0
4,Fermented Foods Mastery,4.5,5,0.5,187,Health & Fitness,Nutrition,Fermented Foods,Kale Brock,English,False,0,3,0,1.500000,12,No,68.98,24.0


## Step 5: Missing Data

In [11]:
data.isna().sum()

Title                            0
Overall_Rating                   0
Best_Rating                      0
Worst_Rating                     0
No_of_Ratings                    0
Category                         0
Subcategory                      0
Topic                         1420
Instructor                       0
Language                         0
SkillsFuture                     0
No_of_Practice_Test              0
No_of_Articles                   0
No_of_Coding_Exercises           0
Video_Duration_Hr              541
No_of_Additional_Resources       0
Bestseller                       0
Price                            0
Discounted_Price              1286
dtype: int64

**Missing Values Reasoning**
<li> "Topic" has missing values because not all courses are assigned a topic. 
<li> "Discounted_Price" has missing values as these courses do not have discounts given. 
<li> With regards to "Video_Duration_Hr", we have to further investigate the reason behind the missing values. 

### Missing Values: Video Duration

In [12]:
pd.set_option("display.max_rows", None)
pd.DataFrame(data.loc[data["Video_Duration_Hr"].isna(), "Title"])

,Title
31,The SEO MCQ practice test
36,"HTML, CSS and Bootstrap Practice Exams"
152,Diploma in Information System Audit (DISA)- ICAI
216,Audit - Practice Questions on Audit Procedures...
256,RMP® Exam Practice Test 1 (6 Exams)
270,Test your knowledge: Kubernetes (Set of 4)
345,"Linux MCQ, University Questions and Online Int..."
373,ISTQB Mobile application testing Practice set
380,E_HANABW_13 || SAP BW 7.5 powered by SAP HANA ...
438,Learn Multiplication of two digit numbers


Upon eyeballing, we can see that the courses with missing video duration values are due to the fact that their nature is of practice tests and exams. Therefore there would be no videos for these types of courses. Hence to tackle this, I will replace all video durations to 0 for all tests and exams.

In [13]:
# Replace test/exam durations with 0 if NA 
data["Video_Duration_Hr"].fillna(0, inplace=True)

# Checking if there has been a decrease in number of missing values
data["Video_Duration_Hr"].isna().sum()

0

### Missing Values: Topics

To handle the missing values in "Topics", I will simply create a new category for them "Unknown"

In [14]:
data["Topic"].fillna("Unknown", inplace = True)
# Checking for number of missing values left
data["Topic"].isna().sum()

0

### Missing Values: Discounted Price

I will fill in the missing discounted prices with its original price since no discounts were given for these courses.

In [15]:
# Replacing price values
data.loc[data["Discounted_Price"].isna(), "Discounted_Price"] = data.loc[data["Discounted_Price"].isna(),"Price"]
# Checking for number of missing values left
data["Discounted_Price"].isna().sum()

0

## Export Data

In [16]:
# Export final dataframe to csv
data.to_csv("./Data/Udemy_Clean.csv")